![alt text](DataKind_orange.png)

### Performs grid search for binary classification of Stabilization and Royalties clauses

In [8]:
import os, io
from collections import Counter
from itertools import product
import re, string
import pandas as pd
import numpy as np

# from sklearn.dummy import DummyClassifier
# from sklearn.linear_model import LogisticRegression
# from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
# , ExtraTreesClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score, recall_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer

import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from textblob import TextBlob
from textstat.textstat import textstat

from IPython.display import display
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_context('notebook', rc={'figure.figsize': (10, 6)}, font_scale=1.5)

In [9]:
grid_search_results = "results_gridsearch/20171108_multiclass_results.txt" 

In [10]:
# spacy is used for Part of Speech tagging and Named Entity Recognition
# spacy is a non-standard python library which can be installed using 'pip install spacy' from the command line
# language models can be downloaded by running 'python -m spacy download <language>' from the command line
import spacy
supported_languages = ['en','fr','es']
language_dict = {'en':'english','fr':'french','es':'spanish'}
nlp_langs = {}
for language in supported_languages:
    nlp_langs[language]  = spacy.load(language)

In [11]:
# Downloaded xls file annotations from resourcecontracts and openlandcontracts
resource_folder = 'contract_data/Contracts_Annotations/resource_contracts/'
land_folder = 'contract_data/Contracts_Annotations/openland_contracts/'
# Most recently downloaded metadata from resourcecontracts.org/contracts
rc_metadata = 'contract_data/resource_contract_2017-08-16.csv' 
# Most recently downloaded metadata from openlandcontracts.org/contracts
ol_metadata = 'contract_data/openland_contract_2017-08-16.csv'
metadata_files = [rc_metadata,ol_metadata]
folders = [resource_folder,land_folder]

In [12]:
labels = ['stabilization','royalties']
n_folds = 5

In [60]:
annotations = pd.DataFrame()
for folder in folders:
    xls_files = [f for f in os.listdir(folder) if f.lower().endswith('.xls')]
    for xls in xls_files:
        temp = pd.read_excel(folder + xls)
        if len(temp) > 0:
            temp['OCID'] = xls[:-4]
            temp['Source'] = folder.split('/')[-2]
            annotations = annotations.append(temp)
print "Number of annotations: " + str(len(annotations))

Number of annotations: 20609


In [61]:
# drop blank and integer annotations and annotations less than 4 words
annotations.dropna(subset=['Annotation Text'],inplace=True)
annotations = annotations[annotations['Annotation Text'].apply(lambda x: type(x)!=int)].copy()
annotations = annotations[annotations['Annotation Text'].apply(lambda x: len(x.split()) > 3)].copy()
print len(annotations)
# If duplicate text appears within the same contract, drop it
annotations.drop_duplicates(['Annotation Text','OCID','Category'],inplace=True)
print len(annotations)

17885
13795


In [62]:
labels = [label.lower() for label in labels]
annotations['label'] = [x.lower() if x.lower() in labels else 'other' for x in annotations['Category']]

In [63]:
# Given some duplicate text with different Category labels, keep the labels that are in our target label list when dropping
sort_num = range(len(labels) + 1)
sort_key = dict(zip(labels,sort_num))
sort_key['other'] = sort_num[-1]
annotations['sort_key'] = [sort_key[x] for x in annotations['label']]
annotations.sort_values(by='sort_key',inplace=True,ascending=True)
annotations.drop_duplicates(['Annotation Text','OCID'],keep='first',inplace=True)
print len(annotations)

13710


In [64]:
label_fixes = pd.read_csv('contract_data/label_fixes/label_fixes.csv')
print len(label_fixes)
annotations = annotations.merge(label_fixes[['OCID','Category','Corrected']],how='left',on=['OCID','Category'])
annotations['Corrected'].fillna('',inplace=True)

14


In [65]:
def update_annotations(row):
    if row['Corrected'] == '':
        return row['label']
    else:
        return row['Corrected']
annotations['label'] = annotations.apply(update_annotations,axis=1)

In [68]:
# Join metadata from contracts repository
metadata = pd.DataFrame()
for filename in metadata_files:
    temp = pd.read_csv(filename)
    metadata = metadata.append(temp)
    
annotations = annotations.merge(metadata[['OCID','Language','Country Name','Resource','Contract Type','Document Type']],how='left',on='OCID')

In [69]:
y = list(annotations['label'])
yunique = list(np.unique(y))
for item in yunique:
    print str("{0:.2f}%".format(100*y.count(item) / float(len(y)))) + " " + item

95.74% other
2.47% royalties
1.79% stabilization


In [70]:
annotations.head(2)

,Category,Topic,Annotation Text,PDF Page Number,Article Reference,OCID,Source,label,sort_key,Corrected,Language,Country Name,Resource,Contract Type,Document Type
0,Stabilization,Legal Rules,The government intends to present to the Natio...,4,Preamble,ocds-591adf-1834156729,resource_contracts,stabilization,0,,en,Iraq,Hydrocarbons,Production or Profit Sharing Agreement,Contract
1,Stabilization,Legal Rules,Le gouvernement s'engage aussi à garantir à SG...,17,Art. 11.1,ocds-591adf-9069819553,resource_contracts,stabilization,0,,fr,Senegal,Gold;Silver,Concession Agreement,Contract


In [71]:
def punctuation_remove(text):
    """
    Mutates and returns text where all punctuation besides underscores,
    are replaced
    """
    punctuation_re = r'[^\w\s_]'
    new_text = nltk.regexp.re.sub(punctuation_re, ' ', text)
    return new_text

def replace_numbers(text):
    ''' 
    Removes all characters but periods, commas and alpha-numeric and 
    returns all numeric values replace with the word numeric_value
    '''
#     allowed = {",", "."," ","%"}.union(string.ascii_letters).union([str(num) for num in range(0,10)])
#     filtered = ''.join([character for character in text if character in allowed])
    wordlist = text.split()
    for i in range(len(wordlist)):
        if '$' in wordlist[i]:
            try:
                int(wordlist[i].split('$')[-1].replace(',','').replace('.','').replace('-','').replace(')','').replace('(','').replace('\'','').replace(';','').replace(':',''))
                wordlist[i] = ' '.join(wordlist[i].split('$')[:-1]) + ' dollarvalue'
            except:
                pass
        elif u'\xb0' in wordlist[i]:
            wordlist[i] == 'degreevalue'
        elif '%' in wordlist[i]:
            try: 
                int(wordlist[i].replace(',','').replace('.','').replace('-','').replace('/','').replace('%','').replace(')','').replace('(','').replace('\'','').replace(';','').replace(':',''))
                wordlist[i] = 'percentvalue'
            except:
                pass
        else:
            try: 
                int(wordlist[i].replace(',','').replace('.','').replace('-','').replace('/','').replace(')','').replace('(','').replace('\'','').replace(';','').replace(':',''))
                wordlist[i] = 'numericvalue'
            except:
                pass
    return ' '.join(wordlist)

def perform_lowercase(text):
    """
    Mutates and returns text where all characters are lowercased
    """
    try:
        new_text = text.lower()
    except:
        new_text = str(text).lower()
    return new_text

def underscore_remove(text):
    '''
    replaces multiple underscores with text fillintheblank
    and single underscore with space
    '''
    double_underscore_re = r'(__[a-zA-Z0-9_]*(__)?)'
    text = nltk.regexp.re.sub(double_underscore_re,'fillintheblank',text)
    return text.replace('_',' ')

def doublespace_remove(text):
    return re.sub(' +',' ',text)

def textblobsent(text):
    '''
    returns the TextBlob polarity and subjectivity
    '''
    text = text.encode('ascii','replace')
    sent = TextBlob(text).sentiment
    return pd.Series([sent.polarity,sent.subjectivity])

def get_avg_wordlength(document):
    wordlengths = [len(word) for word in document.split()]
    if len(wordlengths) == 0:
        return 0
    else:
        return np.mean(wordlengths)

def get_multilingual_pos(text,language):
    if language in nlp_langs.keys():
        try:
            tokens = nlp_langs[language](text)
            tags = [token.pos_ for token in tokens]
            counts = Counter(tags).items()
            countdict = {}
            for key, value in counts:
                countdict[key] = value
            return countdict
        except:
            return {'pos_error':1}
    else:
        return {'unsupported_language':1}

def get_multilingual_entities(row):
    
    text = row['CleanText']
    language = row['Language']
    if language in nlp_langs.keys():
        try:
            doc = nlp_langs[language](text)
            labels = [ent.label_ for ent in doc.ents]
            texts = [ent.text for ent in doc.ents]
            starts = [ent.start_char for ent in doc.ents]
            ends = [ent.end_char for ent in doc.ents]
            textlens = [len(word) for word in texts]
            labellens = [len(label) for label in labels]
            diffs = [textlens[i] - labellens[i] for i in range(len(textlens))]
            diffsum = [sum(diffs[0:i]) for i in range(len(diffs))]

            for i in range(len(labels)):
                text = text[0:starts[i] - diffsum[i]] + 'entity' + labels[i] + text[ends[i] - diffsum[i]:]
            return text
        except:
            return text
    else:
        return text
    
def remove_stopwords(row):
    '''
    Multilingual stopwords removal
    '''
    try:
        language = language_dict[row['Language']]
        text = ' '.join([word for word in row['CleanText'].split(' ') if word not in stopwords.words(language)])
        return text
    except:
        return row['CleanText']
    
def stem_words(row):
    ''' 
    Multilingual word stemmer
    '''
    language = language_dict[row['Language']]
    try:
        stemmer = SnowballStemmer(language)
        text = ' '.join([stemmer.stem(word) for word in row['CleanText_NoStop'].split(' ')])
        return text
    except:
        return row['CleanText_NoStop']
    
def clean_metadata(text):
    if type(text) in [float,int]:
        return text
    elif type(text) == str:
        return text.lower().split(';')[0]
    else:
        text = text.encode('ascii','replace')
        return text.lower().split(';')[0]

In [72]:
def clean_text(df):
    df['Annotation Text'].fillna('',inplace=True)
    df['CleanText'] = df['Annotation Text']
    df['CleanText'] = df.apply(get_multilingual_entities,axis=1)
    func_list = [perform_lowercase,replace_numbers,punctuation_remove,underscore_remove, doublespace_remove]
    for func in func_list:
        df['CleanText'] = df['CleanText'].apply(func)

    return df

def featurize(df):
    
    df['AvgWordLength'] = df['CleanText'].apply(get_avg_wordlength)
    
    df['CleanText_NoStop'] = df.apply(remove_stopwords,axis=1)
    df['CleanText_NoStop_Stemmed'] = df.apply(stem_words,axis=1)
    
    postagcounts = []
    entitycounts = []
    for index, row in df.iterrows():
        postagcounts.append(get_multilingual_pos(row['Annotation Text'],row['Language']))    
    postagdf = pd.DataFrame(postagcounts).fillna(0)
    postagdf.index = df.index
    postagdf.columns = ['postag_' + col for col in postagdf.columns]
    
    # create dummy variables for categoricals
    df['Resource'] = df['Resource'].apply(clean_metadata)
    df['Contract Type'] = df['Contract Type'].apply(clean_metadata)
    df['Document Type'] = df['Document Type'].apply(lambda x: x.lower().split(';')[0])
    dummy_cols = ['Language','Country Name','Resource','Contract Type','Document Type']
    dummies = pd.get_dummies(df[dummy_cols],prefix = dummy_cols)
    # drop lowest least frequent dummy columns for each
    for dummy_col in dummy_cols:
        cols = [col for col in dummies.columns if col.startswith(dummy_col)]
        dummies.drop([col for col, val in dummies[cols].sum().iteritems() if val == dummies[cols].sum().min()],axis=1,inplace=True)
    df.drop(dummy_cols, axis=1,inplace=True)
        
    textblobsentdf = df['CleanText'].apply(textblobsent)
    textblobsentdf.columns = ['TextblobPolarity','TextblobSubjectivity']
    df = pd.concat([df,textblobsentdf,postagdf,dummies],axis=1)
    
    return df

In [73]:
annotations = clean_text(annotations)
annotations = featurize(annotations)

In [74]:
tfidf_vectorizer = TfidfVectorizer(max_df= .6,
                                 min_df= .002, 
                                 stop_words=None,  
                                 use_idf=True, 
                                 ngram_range=(1,5))
tfidf_matrix = tfidf_vectorizer.fit_transform(annotations['CleanText_NoStop_Stemmed'].values.astype('U'))
terms = tfidf_vectorizer.get_feature_names()
tfidf_matrix = tfidf_matrix.todense()
tfidf = pd.DataFrame(tfidf_matrix)
tfidf.index = annotations.index
tfidf.columns = terms
print tfidf.shape

(13710, 8449)


In [75]:
exclude = ['Source','Category','Topic','Annotation Text','CleanText','CleanText_NoStop','CleanText_NoStop_Stemmed',
           'OCID','PDF Page Number','Article Reference',"''",'label','sort_key','Corrected']
features = [str(col) for col in annotations.columns.tolist() if not col in exclude]

In [77]:
X = pd.concat([tfidf,annotations[features]],axis=1)

In [78]:
X.fillna(0,inplace=True)
X = X.rename(columns = {'fit':'fit_feature'})

In [79]:
classifiers = [
#             LogisticRegression(),
#             DecisionTreeClassifier(),
            RandomForestClassifier()
#             ExtraTreesClassifier()
             ]

In [104]:
parameters = [
#         {'penalty': ['l1','l2']      # LogisticRegression
#               ,'C': [.001,.01,.1,1,10,100]
#               ,'class_weight': [None,'balanced']
#               ,'n_jobs': [-1]}
#         ,{'max_depth': [8,10,12,14]}       #DecisionTree
         {'estimator__n_estimators': [500],          # RandomForest
          'estimator__n_jobs': [-1],
            "estimator__max_depth": [None,10,14],
              "estimator__max_features": ['auto',.2,.33],
              "estimator__min_samples_split": [2,10],
              "estimator__min_samples_leaf": [1,50],
              "estimator__bootstrap": [False,True],
              "estimator__class_weight": ['balanced','balanced_subsample']}
#         ,{'max_features':[0.2,0.33,0.5]       #ExtraTrees
#               ,'n_estimators': [50]
#               ,'n_jobs': [-1]
#               ,'max_depth': [3,6,9]}
]

In [105]:
# parameters = [
# #         {'penalty': ['l1','l2']      # LogisticRegression
# #               ,'C': [.001,.01,.1,1,10,100]
# #               ,'class_weight': [None,'balanced']
# #               ,'n_jobs': [-1]}
# #         ,{'max_depth': [8,10,12,14]}       #DecisionTree
#          {'estimator__n_estimators': [10],          # RandomForest
#           'estimator__n_jobs': [-1],
#             "estimator__max_depth": [None],
#               "estimator__max_features": ['auto'],
#               "estimator__min_samples_split": [2],
#               "estimator__min_samples_leaf": [1],
#               "estimator__bootstrap": [True],
#               "estimator__class_weight": ['balanced_subsample']}
# #         ,{'max_features':[0.2,0.33,0.5]       #ExtraTrees
# #               ,'n_estimators': [50]
# #               ,'n_jobs': [-1]
# #               ,'max_depth': [3,6,9]}
# ]

In [107]:
def grid_search(X_all, y_all, classifiers, parameters):
    
    # Runs grid search on given list of classifiers and parameters dictionary
    scores = ['recall_macro','accuracy']
    
    results = []
    for i in range(len(classifiers)):
        model = str(classifiers[i]).split('(')[:1][0]
        print '     ******************************************'
        print '     *** ' + model + ' ***'        
        print '     Tuning hyper-parameters for:' 
        for score in scores:
            print '     ' + score.title()
            clf = GridSearchCV(OneVsRestClassifier(classifiers[i]), parameters[i],cv=n_folds,scoring=score, verbose=10)            
            clf.fit(X_all,y_all)
            y_true, y_pred = y_all, clf.predict(X_all)

            print "Accuracy Score: " + str(accuracy_score(y_true, y_pred))
            print classification_report(y_true, y_pred, target_names=yunique)
            result = [score, clf.scorer_, clf.best_params_, clf.best_score_]
            results.append(result)
            with open(grid_search_results, "a") as myfile:
                myfile.write(str(result))

    return clf, results

In [108]:
clf, results = grid_search(X, y, classifiers, parameters)

     ******************************************
     *** RandomForestClassifier ***
     Tuning hyper-parameters for:
     Recall_Macro
Fitting 5 folds for each of 144 candidates, totalling 720 fits
[CV] estimator__min_samples_leaf=1, estimator__class_weight=balanced, estimator__min_samples_split=2, estimator__max_features=auto, estimator__bootstrap=False, estimator__max_depth=None, estimator__n_estimators=500, estimator__n_jobs=-1 
[CV]  estimator__min_samples_leaf=1, estimator__class_weight=balanced, estimator__min_samples_split=2, estimator__max_features=auto, estimator__bootstrap=False, estimator__max_depth=None, estimator__n_estimators=500, estimator__n_jobs=-1, score=0.747520, total= 1.0min
[CV] estimator__min_samples_leaf=1, estimator__class_weight=balanced, estimator__min_samples_split=2, estimator__max_features=auto, estimator__bootstrap=False, estimator__max_depth=None, estimator__n_estimators=500, estimator__n_jobs=-1 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.1min remaining:    0.0s


[CV]  estimator__min_samples_leaf=1, estimator__class_weight=balanced, estimator__min_samples_split=2, estimator__max_features=auto, estimator__bootstrap=False, estimator__max_depth=None, estimator__n_estimators=500, estimator__n_jobs=-1, score=0.690141, total= 1.0min
[CV] estimator__min_samples_leaf=1, estimator__class_weight=balanced, estimator__min_samples_split=2, estimator__max_features=auto, estimator__bootstrap=False, estimator__max_depth=None, estimator__n_estimators=500, estimator__n_jobs=-1 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  2.2min remaining:    0.0s


[CV]  estimator__min_samples_leaf=1, estimator__class_weight=balanced, estimator__min_samples_split=2, estimator__max_features=auto, estimator__bootstrap=False, estimator__max_depth=None, estimator__n_estimators=500, estimator__n_jobs=-1, score=0.688741, total= 1.0min
[CV] estimator__min_samples_leaf=1, estimator__class_weight=balanced, estimator__min_samples_split=2, estimator__max_features=auto, estimator__bootstrap=False, estimator__max_depth=None, estimator__n_estimators=500, estimator__n_jobs=-1 


...

[CV]  estimator__min_samples_leaf=1, estimator__class_weight=balanced_subsample, estimator__min_samples_split=10, estimator__max_features=0.2, estimator__bootstrap=True, estimator__max_depth=14, estimator__n_estimators=500, estimator__n_jobs=-1, score=0.982130, total= 4.4min
[CV] estimator__min_samples_leaf=1, estimator__class_weight=balanced_subsample, estimator__min_samples_split=10, estimator__max_features=0.2, estimator__bootstrap=True, estimator__max_depth=14, estimator__n_estimators=500, estimator__n_jobs=-1 
[CV]  estimator__min_samples_leaf=1, estimator__class_weight=balanced_subsample, estimator__min_samples_split=10, estimator__max_features=0.2, estimator__bootstrap=True, estimator__max_depth=14, estimator__n_estimators=500, estimator__n_jobs=-1, score=0.979212, total= 4.4min
[CV] estimator__min_samples_leaf=1, estimator__class_weight=balanced_subsample, estimator__min_samples_split=10, estimator__max_features=0.2, estimator__bootstrap=True, estimator__max_depth=14, estimator

[CV]  estimator__min_samples_leaf=1, estimator__class_weight=balanced_subsample, estimator__min_samples_split=2, estimator__max_features=0.33, estimator__bootstrap=True, estimator__max_depth=14, estimator__n_estimators=500, estimator__n_jobs=-1, score=0.978848, total= 6.3min
[CV] estimator__min_samples_leaf=1, estimator__class_weight=balanced_subsample, estimator__min_samples_split=2, estimator__max_features=0.33, estimator__bootstrap=True, estimator__max_depth=14, estimator__n_estimators=500, estimator__n_jobs=-1 
[CV]  estimator__min_samples_leaf=1, estimator__class_weight=balanced_subsample, estimator__min_samples_split=2, estimator__max_features=0.33, estimator__bootstrap=True, estimator__max_depth=14, estimator__n_estimators=500, estimator__n_jobs=-1, score=0.980664, total= 6.3min
[CV] estimator__min_samples_leaf=1, estimator__class_weight=balanced_subsample, estimator__min_samples_split=2, estimator__max_features=0.33, estimator__bootstrap=True, estimator__max_depth=14, estimator

[CV]  estimator__min_samples_leaf=50, estimator__class_weight=balanced_subsample, estimator__min_samples_split=10, estimator__max_features=0.33, estimator__bootstrap=True, estimator__max_depth=14, estimator__n_estimators=500, estimator__n_jobs=-1, score=0.930682, total= 5.4min
[CV] estimator__min_samples_leaf=50, estimator__class_weight=balanced_subsample, estimator__min_samples_split=10, estimator__max_features=0.33, estimator__bootstrap=True, estimator__max_depth=14, estimator__n_estimators=500, estimator__n_jobs=-1 
[CV]  estimator__min_samples_leaf=50, estimator__class_weight=balanced_subsample, estimator__min_samples_split=10, estimator__max_features=0.33, estimator__bootstrap=True, estimator__max_depth=14, estimator__n_estimators=500, estimator__n_jobs=-1, score=0.915724, total= 5.4min


[Parallel(n_jobs=1)]: Done 720 out of 720 | elapsed: 3054.0min finished


Accuracy Score: 0.999635302699
               precision    recall  f1-score   support

        other       1.00      1.00      1.00     13126
    royalties       0.99      1.00      0.99       338
stabilization       1.00      1.00      1.00       246

  avg / total       1.00      1.00      1.00     13710



In [111]:
pd.set_option('max_colwidth',500)
resultsdf = pd.DataFrame(results)
resultsdf.columns = ['Score','Scorer','Best Params','Best Score']
resultsdf.to_csv('20171113_grid_search_results.csv')
resultsdf

,Score,Scorer,Best Params,Best Score
0,recall_macro,"make_scorer(recall_score, average=macro, pos_label=None)","{u'estimator__min_samples_leaf': 50, u'estimator__class_weight': u'balanced_subsample', u'estimator__min_samples_split': 10, u'estimator__max_features': 0.2, u'estimator__bootstrap': False, u'estimator__max_depth': None, u'estimator__n_estimators': 500, u'estimator__n_jobs': -1}",0.905581
1,accuracy,make_scorer(accuracy_score),"{u'estimator__min_samples_leaf': 1, u'estimator__class_weight': u'balanced_subsample', u'estimator__min_samples_split': 2, u'estimator__max_features': 0.2, u'estimator__bootstrap': True, u'estimator__max_depth': None, u'estimator__n_estimators': 500, u'estimator__n_jobs': -1}",0.984245
